In [97]:
import pandas as pd
import sqlite3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 300)
pd.set_option('max_colwidth', 20)

df = pd.read_csv('https://raw.githubusercontent.com/ongcp97/BC3407-Team-7/main/appointmentData.csv')
df['Show Up'] = df['Show Up'].replace('Yes', 1).replace('No', 0)
df['Gender'] = df['Gender'].replace('M', 1).replace('F', 0)
df['Day'] = df['Day'].replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                              [1, 2, 3, 4, 5, 6, 7])

df['Waiting Time'] = df['Waiting Time'] * -1  # inverse sign of waiting time
df = df[~df['Age'] < 0]  # remove negative ages
df['Handicap'] = df['Handicap'].replace([2, 3, 4], 1)  # Replace handicap >1 with handicap==1
df['Register Time'] = pd.to_datetime(df['Register Time'])
df = df.rename({'Apointment': 'Appointment'}, axis=1)
df['Appointment'] = pd.to_datetime(df['Appointment'])

df['Appointment Month'] = df['Appointment'].dt.month
df['Appointment Week Number'] = df['Appointment'].dt.week

cols = ['Gender', 'Day', 'Diabetes', 'Drinks', 'HyperTension', 'Handicap', 'Smoker', 'Scholarship', 'Tuberculosis',
        'Sms_Reminder', 'Show Up', 'Appointment Week Number', 'Appointment Month']
df.loc[:, cols] = df.loc[:, cols].astype('category')
# print(df.info())

# Each row is an Appointment ID, sorted by registered time
df = df.sort_values(by=['Register Time'], ascending=True).reset_index(drop=True)
df = df.reset_index(drop=False)
df = df.rename({'index': 'appointment_id'}, axis=1)

# Creating User IDs for Appointments
comparison = ['Age', 'Gender', 'Diabetes', 'Drinks', 'HyperTension', 'Handicap', 'Smoker', 'Scholarship',
              'Tuberculosis']
df['user_id'] = df.groupby(comparison).ngroup()
df = df.sort_values(by=['user_id', 'appointment_id'], ascending=True)

#normalizing tables into (1) users table (2) appointments table

df_users = df[['user_id'] + comparison]
df_users = df_users.drop_duplicates(subset=['user_id'] + comparison, keep='first')

appt_cols = ['appointment_id', 'user_id', 'Register Time', 'Appointment', 'Day', 'Sms_Reminder', 'Waiting Time',
             'Show Up', 'Appointment Month', 'Appointment Week Number']
df_appointments = df[appt_cols]

# df_machine_learning = pd.merge(df_appointments, df_users, on='user_id').drop(['appointment_id', 'user_id'], axis=1)

# df_users.to_csv('df_users.csv',index=False)
# df_appointments.to_csv('df_appointments.csv',index=False)
# df_machine_learning.to_csv('df_machine_learning.csv',index=False)

# Transferring to sqlite
conn = sqlite3.connect('hospital_database.db')
c = conn.cursor()
df_users.to_sql('users', conn, if_exists='replace', index=False)
df_appointments.to_sql('appointments', conn, if_exists='replace', index=False)
conn.commit()


# for machine_learning
df = pd.read_sql('SELECT * FROM appointments left join users using (user_id);',
                 conn,
                 ).drop(['appointment_id', 'user_id'], axis=1)
print(df)


/var/folders/v2/kgjl826x1t79f1gml7fg_08h0000gn/T/ipykernel_5831/2238971926.py:23: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['Appointment Week Number'] = df['Appointment'].dt.week
/Users/chunpu/Desktop/venv_bc3407/lib/python3.9/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


              Register Time          Appointment Day Sms_Reminder  Waiting Time Show Up Appointment Month Appointment Week Number  Age Gender Diabetes Drinks HyperTension Handicap Smoker Scholarship Tuberculosis
0       2013-11-08 11:45...  2014-01-03 00:00...   5            1            56       1                 1                    1       0      0        0      0            0        0      0           0            0
1       2013-11-25 09:58...  2014-01-06 00:00...   1            0            42       1                 1                    2       0      0        0      0            0        0      0           0            0
2       2013-11-26 11:08...  2014-01-07 00:00...   2            1            42       1                 1                    2       0      0        0      0            0        0      0           0            0
3       2013-11-29 11:24...  2014-01-03 00:00...   5            1            35       1                 1                    1       0      0        0  